# Imports & Environment Variables

In [2]:
# Imports & Environment Variables

import pandas as pd
from owlready2 import *
owlready2.JAVA_EXE = "C:/Program Files/Java/jre1.8.0_351/bin/java.exe"

path = "D:/Uni/Semester 5/KRR/Project/Working/"
onto = owlready2.get_ontology(f"file://{path}onto/onto.owl").load()
infer = owlready2.get_ontology(f"file://{path}infers/graph.rdf").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


# Data Cleaner

In [3]:
# Data Cleaner

def DataCleaner(path="D:/Uni/Semester 5/KRR/Project/Working/"):

    # Data Reading

    main_data = pd.read_csv(f"{path}data/video_games.csv")
    comp_data = pd.read_csv(f"{path}data/GamingCompanies-Main.csv")
    plot_data = pd.read_csv(
        f"{path}data/imdb-videogames.csv", index_col=0).dropna()
    sale_data = pd.read_csv(f"{path}data/vgsales.csv").dropna()
    meta_data = pd.read_csv(f"{path}data/games-data.csv").dropna()


    # Data Setting

    main_data.columns = ["name", "isHandheld", "maxPlayers", "isMultiplatform", "isOnline", "genre", "isLicensed", "hasPublisher", "hasSequel", "score", "sales", "price", "console", "ESRBRating", "reRelease", "year", "averageLength", "leisureLength", "medianLength", "polledLength", "rushedLength", "averageLengthComplete",
                         "leisureLengthComplete", "medianLengthComplete", "polledLengthComplete", "rushedLengthComplete", "averageLengthMain", "leisureLengthMain", "medianLengthMain", "polledLengthMain", "rushedLengthMain", "averageLengthMainAndExtra", "leisureLengthMainAndExtra", "medianLengthMainAndExtra", "polledLengthMainAndExtra", "rushedLengthMainAndExtra"]
    main_data = main_data.drop(["score", "year", "sales"], axis=1)

    comp_data.columns = ["name", "mc", "ear", "companyEmployeeCount",
                         "om", "pe", "companyRevenue", "companyOriginCountry"]
    comp_data = comp_data.drop(["mc", "ear", "om", "pe"], axis=1)
    comp_data.name = comp_data.name.str.replace(
        "Frontier Developments", "FrontierDevelopments")
    comp_data.name = comp_data.name.str.replace("Konami Holdings", "Konami")
    comp_data.name = comp_data.name.str.replace(
        "Activision Blizzard", "Activision")

    plot_data.columns = ["name", "url", "year", "certi", "IMDBScore", "votes",
                         "plot", "ac", "ad", "com", "cri", "fam", "fan", "mys", "sci", "thri"]
    plot_data = plot_data.drop(["url", "year", "certi", "votes", "ac",
                                "ad", "com", "cri", "fam", "fan", "mys", "sci", "thri"], axis=1)

    sale_data.columns = ["rankBySales", "name", "plat", "year", "genre",
                         "publi", "NASales", "EUSales", "JPSales", "other", "GlobalSales"]
    sale_data.NASales = sale_data.NASales * 1_000_000
    sale_data.JPSales = sale_data.JPSales * 1_000_000
    sale_data.EUSales = sale_data.EUSales * 1_000_000
    sale_data.GlobalSales = sale_data.GlobalSales * 1_000_000
    sale_data = sale_data.drop(
        ["plat", "year", "genre", "publi", "other"], axis=1)

    meta_data.columns = ["name", "plat", "releaseDate", "metacriticScore", "userScore",
                         "hasDeveloper", "genre", "plas", "metacriticCriticCount", "usersCriticCount"]
    meta_data = meta_data.drop(["plat", "genre", "plas"], axis=1)


    # Data Merging

    lay1_data = pd.merge(main_data, plot_data, how="inner",
                         on="name").drop_duplicates(["name"]).reset_index(drop=True)
    lay2_data = pd.merge(lay1_data, sale_data, how="inner",
                         on="name").drop_duplicates(["name"]).reset_index(drop=True)
    game_data = pd.merge(lay2_data, meta_data, how="inner",
                         on="name").drop_duplicates(["name"]).reset_index(drop=True)


    # Final Data Setting

    game_data[game_data.userScore == "tbd"] = 0
    game_data = game_data[game_data.isHandheld != 0]
    game_data = game_data[game_data["plot"] != "Add a Plot"]
    game_data = game_data[game_data.hasDeveloper != "No info"]
    game_data.genre = game_data.genre.str.replace(
        "Racing / Driving", "Racing", regex=False)
    game_data.genre = game_data.genre.str.replace(
        "Role-Playing (RPG)", "RPG", regex=False)
    game_data.hasPublisher = game_data.hasPublisher.str.replace(
        ",Inc.", "", regex=False)
    game_data.hasDeveloper = game_data.hasDeveloper.str.replace(
        ",Inc.", "", regex=False)
    game_data.releaseDate = pd.to_datetime(game_data.releaseDate)
    game_data = game_data.dropna().sort_values(
        "rankBySales").reset_index(drop=True)

    return game_data, comp_data

# Knowledge Graph Maker

In [4]:
# Knowledge Graph Maker

def KGMaker(path="D:/Uni/Semester 5/KRR/Project/Working/"):

    # Clean Data

    game_data, comp_data= DataCleaner(path)


    # Company Populating

    with onto:
        for i in comp_data.T:
            name = comp_data.iloc[i]["name"]
            companyEmployeeCount = int(comp_data.iloc[i]["companyEmployeeCount"])
            companyRevenue = int(comp_data.iloc[i]["companyRevenue"])
            companyOriginCountry = comp_data.iloc[i]["companyOriginCountry"].strip()

            company = onto.Company(name.strip().replace(" ", ""))
            company.companyName = name
            if companyEmployeeCount != 0:
                company.companyEmployeeCount = companyEmployeeCount
            company.companyRevenue = companyRevenue
            company.companyOriginCountry = companyOriginCountry


    # COnsole Populating

    with onto:
        for console in game_data.console.unique():
            onto.Console(console.strip().replace(" ", ""))
    

    # Game Populating

    with onto:
        for i in game_data.T:

            # Game Object Making and Name assigning

            name = game_data.iloc[i]["name"]
            sName = name.strip().replace(" ", "")

            game = onto.Game(sName)
            game.gameName = name


            # Game Metadata Object making and assigning that to the game object

            genres = game_data.iloc[i]["genre"].split(",")
            isLicensed = bool(game_data.iloc[i]["isLicensed"])
            hasSequel = bool(game_data.iloc[i]["hasSequel"])
            plot = game_data.iloc[i]["plot"].strip()
            ESRBRating = game_data.iloc[i]["ESRBRating"]

            for genre in genres:
                genreObj = IRIS[f"{onto.base_iri}{genre.strip()}"]
                if not game.hasGenre:
                    game.hasGenre = [genreObj]
                else:
                    game.hasGenre.append(genreObj)

            gameMeta = onto.GameMetadata(f"{sName}Metadata")
            gameMeta.isLicensed = isLicensed
            gameMeta.hasSequel = hasSequel
            gameMeta.plot = plot
            gameMeta.ESRBRating = ESRBRating

            game.hasMetadata = gameMeta


            # Game Metics Object making and assigning that to the game object

            IMDBScore = float(game_data.iloc[i]["IMDBScore"])
            metacriticScore = int(game_data.iloc[i]["metacriticScore"])
            metacriticCriticCount = int(game_data.iloc[i]["metacriticCriticCount"])
            userScore = float(game_data.iloc[i]["userScore"])
            usersCriticCount = int(game_data.iloc[i]["usersCriticCount"])
            price = float(game_data.iloc[i]["price"])

            gameMetrics = onto.GameMetrics(f"{sName}Metrics")
            gameMetrics.IMDBScore = IMDBScore
            gameMetrics.metacriticScore = metacriticScore
            gameMetrics.metacriticCriticCount = metacriticCriticCount
            gameMetrics.userScore = userScore
            gameMetrics.usersCriticCount = usersCriticCount
            gameMetrics.price = price

            game.hasMetrics = gameMetrics


            # Game Features Object making and assigning that to the game object

            isHandheld = bool(game_data.iloc[i]["isHandheld"])
            maxPlayers = int(game_data.iloc[i]["maxPlayers"])
            isMultiplatform = bool(game_data.iloc[i]["isMultiplatform"])
            isOnline = bool(game_data.iloc[i]["isOnline"])

            gameFeats = onto.GameFeatures(f"{sName}Features")
            gameFeats.isHandheld = isHandheld
            gameFeats.maxPlayers = maxPlayers
            gameFeats.isMultiplatform = isMultiplatform
            gameFeats.isOnline = isOnline

            game.hasFeatures = gameFeats


            # Game ReleaseData Object making and assigning that to the game object

            console = game_data.iloc[i]["console"].strip().replace(" ", "")
            releaseDate = game_data.iloc[i]["releaseDate"].to_pydatetime().date()
            reRelease = bool(game_data.iloc[i]["reRelease"])

            consoleObj = IRIS[f"{onto.base_iri}{console}"]
            game.hasConsole = consoleObj
            if not consoleObj.isConsoleOf:
                consoleObj.isConsoleOf = [game]
            else:
                consoleObj.isConsoleOf.append(game)

            gameRD = onto.GameReleaseData(f"{sName}ReleaseData")
            gameRD.releaseDate = releaseDate
            gameRD.reRelease = reRelease

            game.hasReleaseData = gameRD


            # Game Length Data Object making and assigning that to the game object

            averageLength = float(game_data.iloc[i]["averageLength"])
            leisureLength = float(game_data.iloc[i]["leisureLength"])
            medianLength = float(game_data.iloc[i]["medianLength"])
            polledLength = int(game_data.iloc[i]["polledLength"])
            rushedLength = float(game_data.iloc[i]["rushedLength"])

            if averageLength or leisureLength or medianLength or polledLength or rushedLength:
                gameLength = onto.GameLength(f"{sName}LengthData")
                if averageLength:
                    gameLength.averageLength = averageLength
                if leisureLength:
                    gameLength.leisureLength = leisureLength
                if medianLength:
                    gameLength.medianLength = medianLength
                if polledLength:
                    gameLength.polledLength = polledLength
                if rushedLength:
                    gameLength.rushedLength = rushedLength

                game.hasLength = gameLength

            
            # Game Completionists Length Data Object making and assigning that to the game object

            averageLengthCompletionists = float(game_data.iloc[i]["averageLengthComplete"])
            leisureLengthCompletionists = float(game_data.iloc[i]["leisureLengthComplete"])
            medianLengthCompletionists = float(game_data.iloc[i]["medianLengthComplete"])
            polledLengthCompletionists = int(game_data.iloc[i]["polledLengthComplete"])
            rushedLengthCompletionists = float(game_data.iloc[i]["rushedLengthComplete"])

            if averageLengthCompletionists or leisureLengthCompletionists or medianLengthCompletionists or polledLengthCompletionists or rushedLengthCompletionists:
                gameLengthCompletionists = onto.GameLengthCompletionists(f"{sName}CompletionistsLengthData")
                if averageLengthCompletionists:
                    gameLengthCompletionists.averageLengthCompletionists = averageLengthCompletionists
                if leisureLengthCompletionists:
                    gameLengthCompletionists.leisureLengthCompletionists = leisureLengthCompletionists
                if medianLengthCompletionists:
                    gameLengthCompletionists.medianLengthCompletionists = medianLengthCompletionists
                if polledLengthCompletionists:
                    gameLengthCompletionists.polledLengthCompletionists = polledLengthCompletionists
                if rushedLengthCompletionists:
                    gameLengthCompletionists.rushedLengthCompletionists = rushedLengthCompletionists

                game.hasLengthCompletionists = gameLengthCompletionists

           
            # Game Main Length Data Object making and assigning that to the game object

            averageLengthMain = float(game_data.iloc[i]["averageLengthMain"])
            leisureLengthMain = float(game_data.iloc[i]["leisureLengthMain"])
            medianLengthMain = float(game_data.iloc[i]["medianLengthMain"])
            polledLengthMain = int(game_data.iloc[i]["polledLengthMain"])
            rushedLengthMain = float(game_data.iloc[i]["rushedLengthMain"])

            if averageLengthMain or leisureLengthMain or medianLengthMain or polledLengthMain or rushedLengthMain:
                gameLengthMain = onto.GameLengthMain(f"{sName}MainLengthData")
                if averageLengthMain:
                    gameLengthMain.averageLengthMain = averageLengthMain
                if leisureLengthMain:
                    gameLengthMain.leisureLengthMain = leisureLengthMain
                if medianLengthMain:
                    gameLengthMain.medianLengthMain = medianLengthMain
                if polledLengthMain:
                    gameLengthMain.polledLengthMain = polledLengthMain
                if rushedLengthMain:
                    gameLengthMain.rushedLengthMain = rushedLengthMain

                game.hasLengthMain = gameLengthMain


            # Game Main And Extra Length Data Object making and assigning that to the game object

            averageLengthMainAndExtra = float(game_data.iloc[i]["averageLengthMainAndExtra"])
            leisureLengthMainAndExtra = float(game_data.iloc[i]["leisureLengthMainAndExtra"])
            medianLengthMainAndExtra = float(game_data.iloc[i]["medianLengthMainAndExtra"])
            polledLengthMainAndExtra = int(game_data.iloc[i]["polledLengthMainAndExtra"])
            rushedLengthMainAndExtra = float(game_data.iloc[i]["rushedLengthMainAndExtra"])

            if averageLengthMainAndExtra or leisureLengthMainAndExtra or medianLengthMainAndExtra or polledLengthMainAndExtra or rushedLengthMainAndExtra:
                gameLengthMainAndExtra = onto.GameLengthMainAndExtra(f"{sName}MainAndExtraLengthData")
                if averageLengthMainAndExtra:
                    gameLengthMainAndExtra.averageLengthMainAndExtra = averageLengthMainAndExtra
                if leisureLengthMainAndExtra:
                    gameLengthMainAndExtra.leisureLengthMainAndExtra = leisureLengthMainAndExtra
                if medianLengthMainAndExtra:
                    gameLengthMainAndExtra.medianLengthMainAndExtra = medianLengthMainAndExtra
                if polledLengthMainAndExtra:
                    gameLengthMainAndExtra.polledLengthMainAndExtra = polledLengthMainAndExtra
                if rushedLengthMainAndExtra:
                    gameLengthMainAndExtra.rushedLengthMainAndExtra = rushedLengthMainAndExtra

                game.hasLengthMainAndExtra = gameLengthMainAndExtra


            # Game Regional Sales Data Objects making and assigning those to the game object

            NASales = float(game_data.iloc[i]["NASales"])
            EUSales = float(game_data.iloc[i]["EUSales"])
            JPSales = float(game_data.iloc[i]["JPSales"])
            GlobalSales = float(game_data.iloc[i]["GlobalSales"])

            if NASales:
                naregion = onto.RegionalData(f"{sName}NorthAmericanMetrics")
                naregion.regionName = "North America"
                naregion.regionalSales = NASales
                if not game.hasRegionalMetrics:
                    game.hasRegionalMetrics = [naregion]
                else:
                    game.hasRegionalMetrics.append(naregion)

            if EUSales:
                euregion = onto.RegionalData(f"{sName}EuropianMetircs")
                euregion.regionName = "Europe"
                euregion.regionalSales = EUSales
                if not game.hasRegionalMetrics:
                    game.hasRegionalMetrics = [euregion]
                else:
                    game.hasRegionalMetrics.append(euregion)

            if JPSales:
                jpregion = onto.RegionalData(f"{sName}JapaneseMetrics")
                jpregion.regionName = "Japan"
                jpregion.regionalSales = JPSales
                if not game.hasRegionalMetrics:
                    game.hasRegionalMetrics = [jpregion]
                else:
                    game.hasRegionalMetrics.append(jpregion)

            if GlobalSales:
                gbregion = onto.RegionalData(f"{sName}GlobalMetircs")
                gbregion.regionName = "Global"
                gbregion.regionalSales = GlobalSales
                if not game.hasRegionalMetrics:
                    game.hasRegionalMetrics = [gbregion]
                else:
                    game.hasRegionalMetrics.append(gbregion)


            # Game Developer Data Object making and assigning that to the game object

            developers = game_data.iloc[i]["hasDeveloper"].split(",")

            for developer in developers:
                hasDeveloper = developer.strip()
                sHasDeveloper = hasDeveloper.replace(" ", "")
                company = IRIS[f"{onto.base_iri}{sHasDeveloper}"]
                if company:
                    if not game.hasDeveloper:
                        game.hasDeveloper = [company]
                    else:
                        game.hasDeveloper.append(company)
                    if not company.isDeveloperOf:
                        company.isDeveloperOf = [game]
                    else:
                        company.isDeveloperOf.append(game)
                else:
                    company = onto.Company(sHasDeveloper)
                    company.companyName = hasDeveloper
                    company.companyOriginCountry = "Not Known"
                    if not game.hasDeveloper:
                        game.hasDeveloper = [company]
                    else:
                        game.hasDeveloper.append(company)
                    if not company.isDeveloperOf:
                        company.isDeveloperOf = [game]
                    else:
                        company.isDeveloperOf.append(game)


            # Game Developer Data Object making and assigning that to the game object

            publishers = game_data.iloc[i]["hasPublisher"].split(",")

            for publisher in publishers:
                hasPublisher = publisher.strip()
                sHasPublisher = hasPublisher.replace(" ", "")
                company = IRIS[f"{onto.base_iri}{sHasPublisher}"]
                if company:
                    if not game.hasPublisher:
                        game.hasPublisher = [company]
                    else:
                        game.hasPublisher.append(company)
                    if not company.isPublisherOf:
                        company.isPublisherOf = [game]
                    else:
                        company.isPublisherOf.append(game)
                else:
                    company = onto.Company(sHasPublisher)
                    company.companyName = hasPublisher
                    company.companyOriginCountry = "Not Known"
                    if not game.hasPublisher:
                        game.hasPublisher = [company]
                    else:
                        game.hasPublisher.append(company)
                    if not company.isPublisherOf:
                        company.isPublisherOf = [game]
                    else:
                        company.isPublisherOf.append(game)


    # Saving the Built Graph

    onto.save(f"{path}graph/graph.rdf", "rdfxml")

In [ ]:
KGMaker()